In [12]:

import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, TimeDistributed, RepeatVector, Dense, BatchNormalization
import numpy as np
import wandb
from wandb.keras import WandbCallback
import random

wandb.init()
config = wandb.config

class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.
        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)

# Parameters for the model and dataset.
config.hidden_size = 128
config.batch_size = 32

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
maxlen = 25
output_len = 10
# All the numbers, plus sign and space for padding.
chars = '0123456789+$-,/.ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuwxyv '
ctable = CharacterTable(chars)


import pandas as pd
df = pd.read_csv("sale_prices.csv")
df = df.fillna(0)
from sklearn.utils import shuffle
df = shuffle(df)


raw_prices = df["Price on site"].tolist()
parced_prices = []

int_min_price = [int(price) for price in df["Min price"]]
int_max_price = [int(price) for price in df["Max price"]]

#MIN_PRICE MODEL

#train data

split_at = len(int_min_price) - len(int_min_price) // 10




In [13]:
#MIN_PRICE MODEL

raw_prices_train = []
parced_prices_train = []

#training data augmentation  
for i in range(1000):  #augmentation x100 times (from ~500 to 500000 ~datapoints)
    for idx in range(split_at):
        raw_price = str(raw_prices[idx])
        parsed_price = str(int_min_price[idx])
        for i in range(len(parsed_price)):
            to_replace = parsed_price[i]
            replace_with = to_replace
            if i == 0: # to do not make first digit as zero
                if str(int_min_price[idx])[0] != '0':
                    replace_with = str(random.randint(1,9))
            else:
                replace_with = str(random.randint(0,9))
            parsed_price = parsed_price.replace(to_replace,replace_with)
            raw_price = raw_price.replace(to_replace,replace_with)

        parsed_price+=' '*(output_len - len(str(int_min_price[idx])))
        parced_prices_train.append(parsed_price)
        raw_prices_train.append(raw_price)
        
#val data
raw_prices_val = []
parced_prices_val = []
for idx in range(split_at,len(raw_prices)-1):
    raw_price = str(raw_prices[idx])
    parsed_price = str(int_min_price[idx])

    parsed_price+=' '*(output_len - len(str(int_min_price[idx])))
    parced_prices_val.append(parsed_price)
    raw_prices_val.append(raw_price)
        
        
print('Total train prices:', len(raw_prices_train))
print('Total validation prices:', len(raw_prices_val))

print('Vectorization...')
x_train = np.zeros((len(raw_prices_train), maxlen, len(chars)), dtype=np.bool)
y_train = np.zeros((len(parced_prices_train), output_len, len(chars)), dtype=np.bool)
for i, sentence in enumerate(raw_prices_train):
    x_train[i] = ctable.encode(str(sentence), maxlen)
for i, sentence in enumerate(parced_prices_train):
    y_train[i] = ctable.encode(str(sentence), output_len)
    
x_val = np.zeros((len(raw_prices_val), maxlen, len(chars)), dtype=np.bool)
y_val = np.zeros((len(parced_prices_val), output_len, len(chars)), dtype=np.bool)
for i, sentence in enumerate(raw_prices_val):
    x_val[i] = ctable.encode(str(sentence), maxlen)
for i, sentence in enumerate(parced_prices_val):
    y_val[i] = ctable.encode(str(sentence), output_len)

model = Sequential()
model.add(LSTM(config.hidden_size, input_shape=(maxlen, len(chars))))
model.add(RepeatVector(output_len))

model.add(BatchNormalization())

model.add(LSTM(config.hidden_size, return_sequences=True))
model.add(TimeDistributed(Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
             metrics=['accuracy'])
model.summary()

# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 5):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=config.batch_size,
              epochs=1,
              validation_data=(x_val, y_val),callbacks=[WandbCallback()])
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print('☑', end=' ')
        else:
            print('☒', end=' ')
        print('G',guess, end='\n')

Total train prices: 575000
Total validation prices: 62
Vectorization...
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 128)               100864    
_________________________________________________________________
repeat_vector_4 (RepeatVecto (None, 10, 128)           0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 10, 128)           512       
_________________________________________________________________
lstm_9 (LSTM)                (None, 10, 128)           131584    
_________________________________________________________________
time_distributed_4 (TimeDist (None, 10, 68)            8772      
Total params: 241,732
Trainable params: 241,476
Non-trainable params: 256
_________________________________________________________________

---------------------------------------

KeyboardInterrupt: 

In [14]:
print('Min price prediction ----------------------------------------')
for ind in range(len(x_val)):
    rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
    preds = model.predict_classes(rowx, verbose=0)
    q = ctable.decode(rowx[0])
    correct = ctable.decode(rowy[0])
    guess = ctable.decode(preds[0], calc_argmax=False)
    print('Q', q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
        print('☑', end=' ')
    else:
        print('☒', end=' ')
    print('G',guess, end='\n')


Min price prediction ----------------------------------------
Q $19,900                   T 19900      ☑ G 19900     
Q $849,000                  T 849000     ☑ G 849000    
Q $350,000                  T 350000     ☑ G 350000    
Q $189,900                  T 189900     ☑ G 189900    
Q $290,000                  T 290000     ☑ G 290000    
Q $135,000                  T 135000     ☑ G 135000    
Q $988,000                  T 988000     ☑ G 988000    
Q $2,399,000                T 2399000    ☑ G 2399000   
Q $4,900,000                T 4900000    ☑ G 4900000   
Q $685,000                  T 685000     ☑ G 685000    
Q $1,725,000                T 1725000    ☑ G 1725000   
Q $1,000,000                T 1000000    ☑ G 1000000   
Q $1,480,000                T 1480000    ☑ G 1480000   
Q $996,904                  T 996904     ☑ G 996904    
Q $735,000                  T 735000     ☑ G 735000    
Q $1,175,000                T 1175000    ☑ G 1175000   
Q $1,399,000                T 1399000    ☑

In [15]:

import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, TimeDistributed, RepeatVector, Dense, BatchNormalization
import numpy as np
import wandb
from wandb.keras import WandbCallback
import random

wandb.init()
config = wandb.config

class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.
        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)

# Parameters for the model and dataset.
config.hidden_size = 128
config.batch_size = 32

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
maxlen = 25
output_len = 10
# All the numbers, plus sign and space for padding.
chars = '0123456789+$-,/.ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuwxyv '
ctable = CharacterTable(chars)


import pandas as pd
df = pd.read_csv("sale_prices.csv")
df = df.fillna(0)
from sklearn.utils import shuffle
df = shuffle(df)


raw_prices = df["Price on site"].tolist()
parced_prices = []

int_min_price = [int(price) for price in df["Min price"]]
int_max_price = [int(price) for price in df["Max price"]]

#MIN_PRICE MODEL

#train data
#augmentation v2

split_at = len(int_min_price) - len(int_min_price) // 10




In [16]:
#MAX_PRICE MODEL

#train data
#augmentation v2

split_at = len(int_max_price) - len(int_max_price) // 10

raw_prices_train = []
parced_prices_train = []

for i in range(1000):  #augmentation x1000 times (from ~500 to 500000 ~datapoints)
    for idx in range(split_at):
        raw_price = str(raw_prices[idx])
        parsed_price = str(int_max_price[idx])
        for i in range(len(parsed_price)):
            to_replace = parsed_price[i]
            replace_with = to_replace
            if i == 0: # to do not make first digit as zero
                if str(int_max_price[idx])[0] != '0':
                    replace_with = str(random.randint(1,9))
            else:
                replace_with = str(random.randint(0,9))
            parsed_price = parsed_price.replace(to_replace,replace_with)
            raw_price = raw_price.replace(to_replace,replace_with)

        parsed_price+=' '*(output_len - len(str(int_max_price[idx])))
        parced_prices_train.append(parsed_price)
        raw_prices_train.append(raw_price)
        
#val data
raw_prices_val = []
parced_prices_val = []
for idx in range(split_at,len(raw_prices)-1):
    raw_price = str(raw_prices[idx])
    parsed_price = str(int_max_price[idx])

    parsed_price+=' '*(output_len - len(str(int_max_price[idx])))
    parced_prices_val.append(parsed_price)
    raw_prices_val.append(raw_price)
        
        
print('Total train prices:', len(raw_prices_train))
print('Total validation prices:', len(raw_prices_val))

print('Vectorization...')
x_train = np.zeros((len(raw_prices_train), maxlen, len(chars)), dtype=np.bool)
y_train = np.zeros((len(parced_prices_train), output_len, len(chars)), dtype=np.bool)
for i, sentence in enumerate(raw_prices_train):
    x_train[i] = ctable.encode(str(sentence), maxlen)
for i, sentence in enumerate(parced_prices_train):
    y_train[i] = ctable.encode(str(sentence), output_len)
    
x_val = np.zeros((len(raw_prices_val), maxlen, len(chars)), dtype=np.bool)
y_val = np.zeros((len(parced_prices_val), output_len, len(chars)), dtype=np.bool)
for i, sentence in enumerate(raw_prices_val):
    x_val[i] = ctable.encode(str(sentence), maxlen)
for i, sentence in enumerate(parced_prices_val):
    y_val[i] = ctable.encode(str(sentence), output_len)

max_price_model = Sequential()
max_price_model.add(LSTM(config.hidden_size, input_shape=(maxlen, len(chars))))
max_price_model.add(RepeatVector(output_len))

max_price_model.add(BatchNormalization())

max_price_model.add(LSTM(config.hidden_size, return_sequences=True))
max_price_model.add(TimeDistributed(Dense(len(chars), activation='softmax')))
max_price_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
             metrics=['accuracy'])
max_price_model.summary()

# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(2):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    max_price_model.fit(x_train, y_train,
              batch_size=config.batch_size,
              epochs=1,
              validation_data=(x_val, y_val),callbacks=[WandbCallback()])
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = max_price_model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print('☑', end=' ')
        else:
            print('☒', end=' ')
        print('G',guess, end='\n')


Total train prices: 575000
Total validation prices: 62
Vectorization...
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 128)               100864    
_________________________________________________________________
repeat_vector_5 (RepeatVecto (None, 10, 128)           0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 10, 128)           512       
_________________________________________________________________
lstm_11 (LSTM)               (None, 10, 128)           131584    
_________________________________________________________________
time_distributed_5 (TimeDist (None, 10, 68)            8772      
Total params: 241,732
Trainable params: 241,476
Non-trainable params: 256
_________________________________________________________________

---------------------------------------

In [19]:
print('Max price prediction ----------------------------------------')
for ind in range(len(x_val)):
    rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
    preds = max_price_model.predict_classes(rowx, verbose=0)
    q = ctable.decode(rowx[0])
    correct = ctable.decode(rowy[0])
    guess = ctable.decode(preds[0], calc_argmax=False)
    print('Q', q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
        print('☑', end=' ')
    else:
        print('☒', end=' ')
    print('G',guess, end='\n')

Max price prediction ----------------------------------------
Q $500,000                  T 500000     ☑ G 500000    
Q $1,200,000                T 1200000    ☑ G 1200000   
Q $575,000                  T 575000     ☑ G 575000    
Q $3,200,000                T 3200000    ☑ G 3200000   
Q $1,699,999                T 1699999    ☑ G 1699999   
Q $19,500,000               T 19500000   ☑ G 19500000  
Q $290,000                  T 290000     ☑ G 290000    
Q $3,784,995+               T 0          ☑ G 0         
Q $229,000                  T 229000     ☑ G 229000    
Q $364,999                  T 364999     ☑ G 364999    
Q $1,100,000                T 1100000    ☑ G 1100000   
Q Est. $85,442              T 85442      ☑ G 85442     
Q $3,350,000                T 3350000    ☑ G 3350000   
Q $4,248,105                T 4248105    ☒ G 4248155   
Q $100,000                  T 100000     ☑ G 100000    
Q $89,900                   T 89900      ☑ G 89900     
Q $275,000                  T 275000     ☑